In [34]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import ollama
import http.client
http.client._MAXHEADERS = 1000

In [35]:
ollama_api = "http://localhost:11434/api/chat"
headers = {"Content-type":"application/json"}
model = "llama2:7b"

In [36]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
}


class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script","style","img","input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
        
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        

In [ ]:
web = Website("https://www.apple.com/")
print(web.get_contents())
print("---------------------------------------------------------------------------")
web.links

Webpage Title:
Buy Cosmetics Products & Beauty Products Online in India at Best Price | Nykaa
Webpage Contents:
categories
brands
*
#
Popular
Luxe
Only At Nykaa
New Launches
nykaa fashion
beauty advice
Sign in
Flat 15% Off
On Acne Fighting Heroes!
Cleansing Gel
Spot Corrector
Gel Moisturizer
Serum
Ends In:
03
h
11
m
47
s
Ends In:
03
h
11
m
47
s
Flat 10% Off On Lips
On ₹6000: Compact & Glow Toner
15% Off + Gift!
Winning Hair Growth Serum
NEW LAUNCH ALERT
24H Volume, Zero Smudge
Spin The Wheel
Free Gift On All Orders
Foundation @ 15% Off
Tested on Indian Skin Tones
Get Upto 15% Off
Viral Hair Wax Stick & more
Up To 35% Off
Glossiest Hair of Your Dreams!
Up To 30% Off
On Makeup Bestsellers!
Nykaa - The Online Beauty Store
How may we help you


---------------------------------------------------------------------------


['/?root=logo',
 '/',
 '/',
 '/',
 '/',
 '/',
 '/',
 'https://www.nykaa.com/brands/himalaya-herbals/c/680?transaction_id=7b01cf3690847522a63bc036db947df9',
 'https://www.nykaa.com/brands/himalaya-herbals/c/680?transaction_id=b4d983a5c9e2fa9b39aa625d89a48f74',
 'https://www.nykaa.com/himalaya-acne-clear-face-cleansing-gel/p/21187972?transaction_id=fa2d9e3a8d58c894b40c244e13bd1022',
 'https://www.nykaa.com/himalaya-acne-clear-treatment-pen/p/21187969?transaction_id=24bee8cedb0b20edba9ddb105e7a8336',
 'https://www.nykaa.com/himalaya-acne-clear-moisturizing-gel/p/21187971?transaction_id=2eb6fff807069bf35f19d61bd44ebf4e',
 'https://www.nykaa.com/himalaya-acne-clear-scar-removal-serum/p/21187970?transaction_id=83f39d08a9790775672532c4d9f8ed2e',
 'https://www.nykaa.com/buy-1-get-1-free-on-nykaa-cosmetics-lips-bestsellers/c/84514?transaction_id=80e5356112c5ce73888d11de9874f471',
 'https://www.nykaa.com/brands/bath-and-body-works/c/705?transaction_id=c3c223fcf72fbd7c7423f791c5536fbb',
 'http://

In [38]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
    You have to decide which of the links would be most relevant to include in a brochure about the company, \
    such as links to an About Page, or a Company page, or Careers/Jobs page. \n"
link_system_prompt += "You should respond in JSON as this example:"
link_system_prompt += """"
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}

"""

In [39]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.     You have to decide which of the links would be most relevant to include in a brochure about the company,     such as links to an About Page, or a Company page, or Careers/Jobs page. 
You should respond in JSON as this example:"
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}




In [40]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [41]:
payload = {
    "model":model,
    "messages":[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(web)}
      ],
    "stream":False
}


def get_links(url):
    web = Website(url)
    response = requests.post(ollama_api, json=payload, headers=headers)
    result = response.json()
    print(result["message"]["content"])
    # Parse the JSON content from the model's response
    try:
        links_json = json.loads(result["message"]["content"])
    except Exception as e:
        print("Error parsing JSON from model response:", e)
        links_json = {"links": []}
    return links_json

In [42]:
huggingface = Website("https://www.apple.com/")
huggingface.links

['/',
 '/us/shop/goto/store',
 '/mac/',
 '/ipad/',
 '/iphone/',
 '/watch/',
 '/apple-vision-pro/',
 '/airpods/',
 '/tv-home/',
 '/entertainment/',
 '/us/shop/goto/buy_accessories',
 'https://support.apple.com/?cid=gn-ols-home-hp-tab',
 '/us/search',
 '/us/shop/goto/bag',
 '#footnote-1',
 '/us/shop/goto/store',
 '/iphone/',
 '/iphone/',
 '/us/shop/goto/buy_iphone',
 '/macbook-air/',
 '/macbook-air/',
 '/us/shop/goto/buy_mac/macbook_air',
 '/us-edu/shop/goto/edu_store',
 '/us-edu/shop/goto/edu_store',
 '/ipad/',
 '/ipad/',
 '/us/shop/goto/buy_ipad',
 '/macbook-pro/',
 '/macbook-pro/',
 '/us/shop/goto/buy_mac/macbook_pro',
 '/apple-watch-series-10/',
 '/apple-watch-series-10/',
 '/us/shop/goto/buy_watch/apple_watch_series_10',
 '/airpods-pro/hearing-health/',
 '/airpods-pro/hearing-health/',
 '/us/shop/goto/buy_airpods/airpods_pro_2',
 '/us/shop/goto/trade_in',
 '/us/shop/goto/trade_in',
 '/apple-card/',
 '/apple-card/',
 'https://card.apple.com/apply/application?referrer=cid%3Dapy-200-10

In [43]:
get_links("https://www.apple.com/")

Here are the relevant web links for a brochure about NYKAA based on the provided list of links:

{
    "links": [
        {"type": "about page", "url": "https://www.nykaa.com/about"},
        {"type": "careers page", "url": "https://www.nykaa.com/careers"}
    ]
}

Explanation:

* The "about" page link is relevant as it provides information about the company's history, mission, and values.
* The "careers" page link is relevant as it provides information about job opportunities at NYKAA and how to apply.

Note that the other links in the list are not relevant for a brochure about NYKAA, such as the links to specific product pages or promotions.
Error parsing JSON from model response: Expecting value: line 1 column 1 (char 0)


{'links': []}

# Make the brochure

In [44]:
def get_all_details(url):
    result = "Landing Page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found Links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [45]:
print(get_all_details("https://www.apple.com/"))

Here are the relevant web links for a brochure about NYKAA based on the list of links provided:

{
    "links": [
        {"type": "About Page", "url": "https://www.nykaa.com/about"},
        {"type": "Careers Page", "url": "https://www.nykaa.com/careers"}
    ]
}

Explanation:

* The About page provides information about the company's history, mission, and values. It is relevant to include this link in a brochure about NYKAA.
* The Careers page lists current job openings and provides information about the company's culture and work environment. It is relevant to include this link in a brochure about NYKAA for job seekers who are interested in working for the company.
Error parsing JSON from model response: Expecting value: line 1 column 1 (char 0)
Found Links: {'links': []}
Landing Page:
Webpage Title:
Apple
Webpage Contents:
Apple
Apple
Store
Mac
iPad
iPhone
Watch
Vision
AirPods
TV & Home
Entertainment
Accessories
Support
0
+
We’re donating $10 to the National Park Foundation for eac

In [46]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [47]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [48]:
get_brochure_user_prompt("Apple", "https://www.apple.com/")

Here are the relevant links from the website <https://www.nykaa.com/> that could be included in a brochure about the company:

1. "/about" - The About page provides information on the company's history, mission, and values.
2. "/brands" - This link leads to a list of all the brands available on the platform, which could be useful for a brochure highlighting the variety of products offered by NYKAA.
3. "/careers" - The Careers page lists job openings and provides information on the company culture and benefits.
4. "/himalaya-herbals" - This link goes to the Himalaya Herbals brand page, which could be included in a brochure if the company wants to highlight their acne treatment products.
5. "/spin-the-wheel" - The Spin the Wheel link leads to a game that provides a fun way for users to explore the platform and discover new products.
6. "/brands/c/1251" - This link goes to the Clinique brand page, which could be included in a brochure if the company wants to highlight their skincare produ

'You are looking at a company called: Apple\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding Page:\nWebpage Title:\nApple\nWebpage Contents:\nApple\nApple\nStore\nMac\niPad\niPhone\nWatch\nVision\nAirPods\nTV & Home\nEntertainment\nAccessories\nSupport\n0\n+\nWe’re donating $10 to the National Park Foundation for each purchase made at Apple using Apple\xa0Pay through 8/29.\n1\nShop\xa0now\niPhone\nMeet the iPhone 16 family.\nLearn more\nShop iPhone\nBuilt for Apple Intelligence.\nMacBook Air\nSky blue color.\nSky high performance with M4.\nLearn more\nBuy\nBuilt for Apple Intelligence.\nBuy Mac or\xa0iPad for\xa0college\nwith education savings\nChoose AirPods or an eligible accessory\n2\nShop\niPad\nTouch, draw, and type on one magical device.\nLearn more\nShop iPad\nBuilt for Apple Intelligence.\nMacBook Pro\nA work of smart.\nLearn more\nBuy\nBuilt for Apple\xa0Intelligence.\nApple

In [51]:
def create_brochure(company_name, url):
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        "stream": False
    }
    response = requests.post(ollama_api, json=payload, headers=headers)
    result = response.json()
    display(Markdown(result["message"]["content"]))
    

In [52]:
create_brochure("Apple", "https://www.apple.com/-")

Here are the relevant links from the list that you provided for a brochure about NYKAA:

{
    "links": [
        {"type": "about page", "url": "https://www.nykaa.com/about"},
        {"type": "careers page", "url": "https://www.nykaa.com/careers"}
    ]
}

The other links seem to be relative links or internal links within the NYKAA website, and do not provide any external information about the company.
Error parsing JSON from model response: Expecting value: line 1 column 1 (char 0)
Found Links: {'links': []}


Apple Inc. is a multinational technology company that designs, manufactures, and markets consumer electronics, computer software, and online services. Based in California, the company was founded in 1976 by Steve Jobs, Steve Wozniak, and Ronald Wayne. Apple is known for its innovative products such as the Macintosh computer, iPod, iPhone, and iPad, as well as its software platforms such as iOS, macOS, watchOS, and tvOS. The company operates in several segments, including the iPhone segment, which accounts for the majority of its revenue.

Apple culture is centered around innovation and creativity, with a strong focus on employee satisfaction and well-being. The company offers various benefits such as on-site fitness classes, medical care, and mental health resources to support employees' well-being. Apple also has a commitment to diversity and inclusion, with programs aimed at fostering a workplace culture that is inclusive and respectful of all individuals.

Apple customers are tech-savvy individuals who value innovation, design, and ease of use. The company offers a wide range of products and services catering to various customer needs, including Mac computers, iPads, iPhones, Apple Watches, AirPods, TVs, and home entertainment systems. Customers can shop online or in-store, with many locations offering workshops and events for customers to learn more about their products.

Apple offers a variety of career opportunities in fields such as engineering, design, marketing, and operations. The company emphasizes innovation and creativity, providing employees with resources to develop new ideas and solutions. Apple also values diversity and inclusivity, with a commitment to hiring candidates from underrepresented groups.

Apple's investor relations focus on maintaining strong financial performance, with a long-term strategy of sustainable growth. The company engages with shareholders through regular earnings calls, analyst meetings, and investor events. Apple also provides updates on its supply chain innovations, environmental initiatives, and corporate social responsibility programs.

In conclusion, Apple is a technology company that values innovation, creativity, diversity, and inclusivity in all aspects of its operations. From products to culture, the company prioritizes customer satisfaction and employee well-being while maintaining strong financial performance. Whether you are a customer, investor, or job seeker, Apple offers opportunities for growth and engagement with its cutting-edge technology and forward-thinking approach.